<a href="https://colab.research.google.com/github/Raaj-4320/Accident-analysis/blob/main/accident_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [353]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
import plotly.graph_objs as go
import plotly.express as px
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC

In [354]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [355]:
df = pd.read_csv("drive/MyDrive/Datasets/Train.csv")

# #EDA

# Checking for number of missing and unique values in dataset

In [356]:
# First result: Total null values for each feature
null_counts = df.isnull().sum()
null_df = pd.DataFrame(null_counts, columns=["Missing Values"])

# Second result: Total unique values for each feature
unique_values = df.nunique().reset_index()
unique_values.columns = ["Feature", "Unique Values"]

# Combine the two DataFrames
combined_df = pd.merge(unique_values, null_df, left_on="Feature", right_index=True)

# Display the combined DataFrame
print(combined_df)

              Feature  Unique Values  Missing Values
0              INDEX_          15000               0
1              ACCNUM           3822            3698
2                YEAR             13               0
3                DATE           3082               0
4                TIME           1276               0
5             STREET1           1547               0
6             STREET2           2344            1343
7              OFFSET            339           13072
8          ROAD_CLASS              9             357
9            DISTRICT              4              14
10            WARDNUM             71               0
11           LATITUDE           3475               0
12          LONGITUDE           3901               0
13           LOCCOORD              7              90
14             ACCLOC              9            5450
15           TRAFFCTL             10              29
16         VISIBILITY              8              14
17              LIGHT              9          

# #INDEX_

In [357]:
# Removing INDEX_ column as it has no relavance to the analysis

In [358]:
df.drop('INDEX_', axis=1, inplace=True)

# #ACCNUM

In [359]:
# Removing those ACCCNUMS which are non-fatal

to_drop = df[(df['ACCNUM'].isna()) & (df['ACCLASS'] == 'Non-Fatal Injury') ].index

df = df.drop(to_drop, axis=0)
# to_drop

In [360]:
'''
ACCNUM is an important becuase it is representative of one accident incident but, we have null values in the columns
So, we are proceeding with creating new ACC_NUM of our own, wich will be decided on the parameters likem, TIME, DATE, YEAR
and STEET1, we will give 1 unique number to those records which have same data in these columns it means that if parties
involved in an accident has same TIME, DATE, YEAR and STREET1, it determines that the other person matching the
same parameteres was involved in the same accident and one unique number will be allocated to that accident.

Which is ultimately same as organicaly given accident id.
'''

'\nACCNUM is an important becuase it is representative of one accident incident but, we have null values in the columns \nSo, we are proceeding with creating new ACC_NUM of our own, wich will be decided on the parameters likem, TIME, DATE, YEAR \nand STEET1, we will give 1 unique number to those records which have same data in these columns it means that if parties \ninvolved in an accident has same TIME, DATE, YEAR and STREET1, it determines that the other person matching the \nsame parameteres was involved in the same accident and one unique number will be allocated to that accident.\n\nWhich is ultimately same as organicaly given accident id.\n'

In [361]:
# Impute those ACCNUM nulls which ahve ACCLASS as FATAL
one_accident = df[df.duplicated(subset=['YEAR','DATE','TIME','STREET1'], keep=False)]

# Group by the condition and assign a unique index ID
one_accident['ACC_NUM'] = one_accident.groupby(['YEAR','DATE','TIME','STREET1']).ngroup()

# Merge the unique index IDs back to the original DataFrame
df = pd.merge(df, one_accident[['ACC_NUM']], left_index=True, right_index=True, how='left')

<ipython-input-361-b6bd90eac855>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_accident['ACC_NUM'] = one_accident.groupby(['YEAR','DATE','TIME','STREET1']).ngroup()


In [362]:
'''
Some records will left as it is, as it doesn't match with our condition, becase those accidents have no other records
matching the requirement we provided so, if they are FATAL then they are important becuase we have less values for FATAL which
our target variable, but if we have Non-Fatal Injury from ACCLASS then we can drop those records because, we hace more than 7x
data of it as compared to FATAL.
'''

"\nSome records will left as it is, as it doesn't match with our condition, becase those accidents have no other records\nmatching the requirement we provided so, if they are FATAL then they are important becuase we have less values for FATAL which \nour target variable, but if we have Non-Fatal Injury from ACCLASS then we can drop those records because, we hace more than 7x \ndata of it as compared to FATAL.\n"

In [363]:
df[(df["ACC_NUM"].isna()) & (df["ACCLASS"] == "Fatal")]

,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM
659,905761.0,2006,2006/05/30 04:00:00+00,2328,ROSEDALE VALLEY RD,BAYVIEW AVE,NaN,Minor Arterial,Toronto and East York,1113,43.671645,-79.366090,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,NaN,East,Motorcycle,Going Ahead,Lost control,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,71,Cabbagetown-South St.James Town,71,Cabbagetown-South St.James Town (71),D51,660,NaN
880,916024.0,2006,2006/07/30 04:00:00+00,2155,F G GARDINER XY Ramp W,GARDINER W S KINGSWAY RAMP,NaN,NaN,Toronto and East York,4,43.634645,-79.471590,Exit Ramp Westbound,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,20 to 24,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D11,881,NaN
1141,930890.0,2006,2006/10/18 04:00:00+00,2136,OLD FINCH AVE,REESOR RD,NaN,Collector,Scarborough,25,43.824745,-79.190590,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,20 to 24,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,Morningside Heights,131,Rouge (131),D42,1142,NaN
2558,987376.0,2007,2007/08/30 04:00:00+00,2252,F G GARDINER XY W,JAMESON AVE,NaN,NaN,Toronto and East York,4,43.632745,-79.433790,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D14,2559,NaN
2916,987500.0,2007,2007/09/02 04:00:00+00,48,SHEPPARD AVE W,SENTINEL RD,NaN,Major Arterial,North York,6,43.743645,-79.491790,Intersection,NaN,Traffic Signal,Clear,Dark,Dry,Fatal,SMV Other,Driver,40 to 44,Fatal,NaN,East,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,Downsview,26,Downsview-Roding-CFB (26),D31,2917,NaN
3903,1039861.0,2008,2008/05/24 04:00:00+00,30,BAYVIEW Aven,BLOOR Ramp,NaN,NaN,Toronto and East York,11,43.677146,-79.367893,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,21.0,North,Motorcycle,Going Ahead,Exceeding Speed Limit,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,98,Rosedale-Moore Park,98,Rosedale-Moore Park (98),D53,3904,NaN
3990,1061341.0,2008,2008/09/28 04:00:00+00,545,WILSON AVE,HIGHWAY 400 S,NaN,Major Arterial,Etobicoke York,7,43.718145,-79.521090,Intersection,At Intersection,No Control,Clear,Dark,Wet,Fatal,SMV Other,Driver,55 to 59,Fatal,42.0,East,"Automobile, Station Wagon",Going Ahead,Exceeding Speed Limit,Had Been Drinking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,154,Oakdale-Beverley Heights,26,Downsview-Roding-CFB (26),D31,3991,NaN
7524,1276186.0,2012,2012/01/20 05:00:00+00,2214,SENECA HILL DR,DON MILLS RD,NaN,Collector,North York,17,43.790045,-79.355190,Mid-Block,NaN,No Control,Snow,Dark,Packed Snow,Fatal,SMV Other,Driver,85 to 89,Fatal,4.0,West,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,Don Valley Village,47,Don Valley Village (47),D33,7525,NaN
8694,1290105.0,2012,2012/04/20 04:00:00+00,1447,DON VALLEY PARKWAY N,GERRARD ST E,NaN,NaN,Toronto and East York,14,43.665145,-79

In [364]:
df[(df["ACC_NUM"].isna()) & (df["ACCLASS"] == "Non-Fatal Injury")]

,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM
277,8.907670e+05,2006,2006/02/28 05:00:00+00,1754,RENFORTH DR,EGLINTON AVE W,NaN,Major Arterial,Etobicoke York,2,43.666645,-79.587390,Mid-Block,NaN,No Control,Clear,Dark,Dry,Non-Fatal Injury,SMV Other,Driver,55 to 59,Major,NaN,North,"Automobile, Station Wagon",Going Ahead,Other,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,Eringate-Centennial-West Deane,11,Eringate-Centennial-West Deane (11),D22,278,NaN
403,9.102580e+05,2006,2006/06/08 04:00:00+00,2340,KING ST E,ST LAWRENCE ST,NaN,Major Arterial,Toronto and East York,13,43.656145,-79.357490,Mid-Block,NaN,No Control,Rain,Dark,Wet,Non-Fatal Injury,SMV Other,Motorcycle Driver,50 to 54,Major,NaN,East,Motorcycle,Changing Lanes,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,Moss Park,73,Moss Park (73),D51,404,NaN
515,9.054570e+05,2006,2006/05/26 04:00:00+00,2214,BATHURST ST,DAVENPORT RD,NaN,Major Arterial,Toronto and East York,12,43.676745,-79.415790,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Cyclist Collisions,Cyclist,40 to 44,Major,NaN,South,Bicycle,Going Ahead,NaN,NaN,NaN,NaN,NaN,Cyclist loses control and strikes object (pole...,Lost control,Had Been Drinking,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,Casa Loma,96,Casa Loma (96),D13,516,NaN
725,9.065260e+05,2006,2006/06/04 04:00:00+00,1655,BRIMLEY RD,CHARTLAND BLVD S,NaN,Major Arterial,Scarborough,23,43.804745,-79.276690,Mid-Block,At Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,SMV Other,Driver,45 to 49,Major,NaN,South,"Automobile, Station Wagon",Slowing or Stopping,Lost control,Medical or Physical Disability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,129,Agincourt North,129,Agincourt North (129),D42,726,NaN
773,9.151010e+05,2006,2006/07/24 04:00:00+00,1302,LAKE SHORE BLVD W,JAMESON LAKESHORE RAMP,NaN,Major Arterial,Toronto and East York,4,43.631745,-79.432590,Mid-Block,NaN,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Cyclist Collisions,Cyclist,30 to 34,Major,NaN,East,Bicycle,Going Ahead,NaN,NaN,NaN,NaN,NaN,Cyclist loses control and strikes object (pole...,Lost control,Unknown,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D14,774,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11451,4.002973e+09,2014,2014/09/24 04:00:00+00,1130,BANBURY RD,LAWRENCE AVE E,10 m North of,Collector,North York,15,43.735106,-79.357433,Intersection,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,SMV Other,Driver,65 to 69,Major,NaN,South,"Automobile, Station Wagon",Going Ahead,Lost control,Medical or Physical Disability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,Yes,42,Banbury-Don Mills,42,Banbury-Don Mills (42),D33,11452,NaN
11453,4.002973e+09,2014,2014/09/24 04:00:00+00,1151,551 THE WEST MALL,NaN,NaN,Minor Arterial,Etobicoke York,2,43.651831,-79.570003,Mid-Block,Non Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,SMV Other,Driver,75 to 79,Major,NaN,North,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,13,Etobicoke West Mall,13,Etobicoke West Mall (13),D22,11454,NaN
11497,4.003003e+09,2014,2014/09/28 04:00:00+00,1820,W

In [365]:
# Dropping rows where ACC_NUM is null and ACCLASS is Non-Fatal

to_drop2 = df[(df["ACC_NUM"].isna()) & (df["ACCLASS"] == "Non-Fatal Injury")].index

df = df.drop(to_drop2, axis=0)

In [366]:
df.ACC_NUM.max()

3898.0

In [367]:
# Giving a new unique number in the continuetion of previously given numbers, for the records which did not match the requirements before

In [368]:
# Find null values in the 'ACC_NUM' column
null_values_indices = df[df['ACC_NUM'].isnull()].index

# Generate a sequence starting from 4545
sequence = range(3899, 3899 + len(null_values_indices))

# Fill null values with the generated sequence
df.loc[null_values_indices, 'ACC_NUM'] = sequence

# Display the DataFrame with filled null values
df["ACC_NUM"].isna().sum()

0

In [369]:
df[df.ACC_NUM.isna()]

,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM


In [370]:
df.ACC_NUM.max()

3937.0

In [371]:
# We can remove the ACCNUM column now

In [372]:
df.drop('ACCNUM', axis=1, inplace=True)

In [373]:
# Checking ACCLASS's number of FATAL values, whether the values of is same or not

In [374]:
df["ACCLASS"].value_counts()

Non-Fatal Injury    9755
Fatal               2022
Name: ACCLASS, dtype: int64

# #YEAR

In [375]:
# Removing YEAR column as it is not relevant

In [376]:
df.drop("YEAR", axis=1, inplace=True)

# #DATE

In [377]:
# Removig DATE column as it is not relevant

In [378]:
df.drop("DATE", axis=1, inplace=True)

# #TIME

In [379]:
# Keeping time column because it may affect the accident class

In [380]:
df.sample(5)

,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM
10082,1515,1092 KEELE ST,NaN,NaN,Local,Etobicoke York,59,43.679190,-79.470340,Mid-Block,NaN,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,Pedestrian,5 to 9,Major,NaN,South,Other,NaN,NaN,NaN,Pedestrian hit at private driveway,Running onto Roadway,Inattentive,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,Weston-Pelham Park,91,Weston-Pellam Park (91),D12,10083,3271.0
4499,1100,TRETHEWEY DR,BROOKHAVEN DR,NaN,Minor Arterial,Etobicoke York,5,43.696945,-79.493890,Intersection,At Intersection,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Vehicle Owner,unknown,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,30,Brookhaven-Amesbury,30,Brookhaven-Amesbury (30),D12,4500,1413.0
841,1610,DUNDAS ST E,BROADVIEW AVE,NaN,Major Arterial,Toronto and East York,14,43.662445,-79.351390,Intersection,NaN,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Motorcycle Driver,20 to 24,Major,NaN,West,Motorcycle,Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,70,South Riverdale,70,South Riverdale (70),D55,842,296.0
8373,223,ST CLAIR AVE W,OLD WESTON RD,NaN,Major Arterial,Toronto and East York,9,43.673545,-79.462990,Intersection,NaN,Traffic Signal,Clear,"Dark, artificial",Dry,Non-Fatal Injury,SMV Other,Passenger,30 to 34,Minor,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,NaN,NaN,91,Weston-Pelham Park,91,Weston-Pellam Park (91),D11,8374,2860.0
11429,617,1792 BIRCHMOUNT RD,NaN,NaN,Major Arterial,Scarborough,21,43.765433,-79.291316,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Pedestrian,55 to 59,Fatal,3.0,West,NaN,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Inattentive,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126,Dorset Park,126,Dorset Park (126),D41,11430,3669.0


In [381]:
# We can divide time in parts of the day

In [382]:
# Function to map time values to segments of the day
def map_time_to_segment(time):
    if time < 600:
        return 'Midnight'
    elif time < 1200:
        return 'Morning'
    elif time < 1700:
        return 'Afternoon'
    elif time < 2100:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create a new column 'TIME_SEGMENT'
df['TIME_SEGMENT'] = df['TIME'].apply(map_time_to_segment)

In [383]:
df[['TIME','TIME_SEGMENT']]

,TIME,TIME_SEGMENT
0,852,Morning
1,852,Morning
2,915,Morning
3,236,Midnight
4,915,Morning
...,...,...
14960,1903,Evening
14964,1903,Evening
14967,1903,Evening
14980,709,Morning


In [384]:
# we can drop the TIME column

In [385]:
df.drop('TIME', axis=1, inplace=True)

# #STREET1

In [386]:
# Keeping STREET1 as it is
df.STREET1.isna().sum()

0

# #STREET2

In [387]:
# Filling null values in STREET2 with Not Applicabel(N/A)

In [388]:
df['STREET2'].fillna('NA', inplace=True)

In [389]:
df.STREET2.isna().sum()

0

# #ROAD_CLASS

In [390]:
df.ROAD_CLASS.value_counts()

Major Arterial    8423
Minor Arterial    1785
Collector          672
Local              554
Other               25
Expressway           8
Laneway              6
Pending              2
Name: ROAD_CLASS, dtype: int64

In [391]:
df['ROAD_CLASS'].fillna('Major Arterial', inplace=True)

In [392]:
df.ROAD_CLASS.isna().sum()

0

In [393]:
#  Now we can drop those ACCNUM column because we have our own ACC_NUM which has no null values

In [394]:
df.columns

Index(['STREET1', 'STREET2', 'OFFSET', 'ROAD_CLASS', 'DISTRICT', 'WARDNUM',
       'LATITUDE', 'LONGITUDE', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL', 'VISIBILITY',
       'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE', 'INVAGE',
       'INJURY', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT',
       'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT',
       'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
       'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING', 'AG_DRIV',
       'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'DIVISION', 'ObjectId', 'ACC_NUM',
       'TIME_SEGMENT'],
      dtype='object')

# #OFFSET

In [395]:
df.OFFSET.isna().sum()

11438

In [396]:
# OFFSET has large number of missing values therefore removing it

In [397]:
df.drop("OFFSET", axis=1, inplace=True)

# #ROAD_CLASS

In [398]:
# Displaying Roadclass column values and if it's class i non-fatal then removing those clumns

In [399]:
df[(df['ROAD_CLASS'].isna())&(df['ACCLASS'] == "Fatal")]

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT


In [400]:
# There are record where ROAD_CLASS is null and it' ACCLASS is null, for that reason we are imputing road class values with most accured values from that column.

In [401]:
# we dont have any null values in road class where acclass is FATAL, it means we can remove all null records

In [402]:
df = df.dropna(subset=['ROAD_CLASS'])

In [403]:
df.ROAD_CLASS.isna().sum()

0

#  #DISTRICT

In [404]:
# Checking null values of DISCTRICT if they are not of FATAL ACCLASS then removing it

In [405]:
df[(df['DISTRICT'].isna())&(df['ACCLASS'] == "Non-Fatal Injury")]

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
11915,WILLIAM R ALLEN RD,SHEPPARD AVE W,Major Arterial,NaN,6,43.748622,-79.462856,Mid-Block,At/Near Private Drive,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Turning Movement,Truck Driver,25 to 29,None,NaN,East,Truck-Tractor,Turning Right,Failed to Yield Right of Way,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,155,Downsview,26,Downsview-Roding-CFB (26),D32,11916,3625.0,Midnight
11917,WILLIAM R ALLEN RD,SHEPPARD AVE W,Major Arterial,NaN,6,43.748622,-79.462856,Mid-Block,At/Near Private Drive,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Turning Movement,Passenger,30 to 34,Major,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,155,Downsview,26,Downsview-Roding-CFB (26),D32,11918,3625.0,Midnight


In [406]:
# drop_district = df[(df['DISTRICT'].isna()) & (df['ACCLASS'] == "Non-Fatal Injury")].index


drop_district = df[(df["DISTRICT"].isna()) & (df["ACCLASS"] == "Non-Fatal Injury")].index

df = df.drop(drop_district, axis=0)

In [407]:
df['DISTRICT']

0        Toronto and East York
1        Toronto and East York
2                  Scarborough
3        Toronto and East York
4                  Scarborough
                 ...          
14960               North York
14964               North York
14967               North York
14980              Scarborough
14981              Scarborough
Name: DISTRICT, Length: 11775, dtype: object

# #LOCCOORD

In [408]:
# Checking null values of LOCCOORD if they are not of FATAL ACCLASS then removing it

In [409]:
df[(df['LOCCOORD'].isna())&(df['ACCLASS'] == "Fatal")]

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT


In [410]:
df[(df['LOCCOORD'].isna())&(df['ACCLASS'] == "Non-Fatal Injury")]


,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
278,DANFORTH RD,EGLINTON AVE E,Major Arterial,Scarborough,2120,43.737045,-79.24619,NaN,NaN,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Vehicle Owner,unknown,None,NaN,NaN,Other,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,Eglinton East,138,Eglinton East (138),D43,279,55.0,Night
279,DANFORTH RD,EGLINTON AVE E,Major Arterial,Scarborough,2120,43.737045,-79.24619,NaN,NaN,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,unknown,None,NaN,West,"Automobile, Station Wagon",Going Ahead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,Eglinton East,138,Eglinton East (138),D43,280,55.0,Night
280,DANFORTH RD,EGLINTON AVE E,Major Arterial,Scarborough,2120,43.737045,-79.24619,NaN,NaN,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Pedestrian,unknown,Minor,NaN,NaN,Other,Other,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,Eglinton East,138,Eglinton East (138),D43,281,55.0,Night
281,DANFORTH RD,EGLINTON AVE E,Major Arterial,Scarborough,2120,43.737045,-79.24619,NaN,NaN,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Pedestrian,unknown,Major,NaN,NaN,Other,Other,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,Eglinton East,138,Eglinton East (138),D43,282,55.0,Night
1049,DUFFERIN ST,GLEN BELLE CRES,Major Arterial,North York,8,43.722245,-79.45649,NaN,NaN,No Control,Clear,Dark,Wet,Non-Fatal Injury,Pedestrian Collisions,Driver,unknown,None,NaN,NaN,"Automobile, Station Wagon",Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,Yorkdale-Glen Park,31,Yorkdale-Glen Park (31),D32,1050,417.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9869,LAKE SHORE BLVD W,QUEENS AVE,Major Arterial,Etobicoke York,3,43.609660,-79.49026,NaN,NaN,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Driver,50 to 54,Major,NaN,NaN,"Automobile, Station Wagon",Going Ahead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,160,Mimico-Queensway,17,Mimico (includes Humber Bay Shores) (17),D22,9870,3175.0,Afternoon
9871,LAKE SHORE BLVD W,QUEENS AVE,Major Arterial,Etobicoke York,3,43.609660,-79.49026,NaN,NaN,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Vehicle Owner,unknown,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,160,Mimico-Queensway,17,Mimico (includes Humber Bay Shores) (17),D22,9872,3175.0,Afternoon
9882,LAKE SHORE BLVD W,QUEENS AVE,Major Arterial,Etobicoke York,3,43.609660,-79.49026,NaN,NaN,No Control,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Driver,20 to 24,Minor,NaN,NaN,"Automobile, Station Wagon",Making U Turn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,160,Mimico-Queensway,17,Mimico (includes Humber Bay Shores) (17),D22,9883,3175.0,Afternoon
10963,BAYVIEW BLOOR DVP RAMP,BAYVIEW BLOOR DVP N RAMP,Major Arterial,Toronto and East York,14,43.683283,-79.36021,NaN,NaN,Yield Sign,Clear,Dark,Dry,Non-Fatal Injury,SMV Other,Passenger,unknown,Minor,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,57,Broadview North,57

In [411]:
# There are null values in LOCCOORD whicha have ACCLASS as Non-Fatal, we can drop those colums

In [412]:
drop_loccoord = df[(df["LOCCOORD"].isna()) & (df["ACCLASS"] == "Non-Fatal Injury")].index

df = df.drop(drop_loccoord, axis=0)

# #ACCLOC

In [413]:
# Checking values of ACCLOC

In [414]:
df["ACCLOC"].value_counts()

At Intersection          5466
Non Intersection          558
Intersection Related      308
At/Near Private Drive      47
Underpass or Tunnel         5
Private Driveway            5
Overpass or Bridge          5
Name: ACCLOC, dtype: int64

In [415]:
df[(df["ACCLOC"].isna()) & (df["ACCLASS"] == "Fatal")]


,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
5,EGLINTON AVE E,COMMONWEALTH AVE,Major Arterial,Scarborough,2120,43.734945,-79.256190,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Driver,25 to 29,None,NaN,West,"Automobile, Station Wagon",Going Ahead,Other,"Ability Impaired, Alcohol",NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,124,Kennedy Park,138,Eglinton East (138),D41,6,62.0,Midnight
8,EGLINTON AVE E,COMMONWEALTH AVE,Major Arterial,Scarborough,2120,43.734945,-79.256190,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Pedestrian,30 to 34,Minor,NaN,South,Other,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Normal,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,124,Kennedy Park,138,Eglinton East (138),D41,9,62.0,Midnight
9,EGLINTON AVE E,COMMONWEALTH AVE,Major Arterial,Scarborough,2120,43.734945,-79.256190,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Pedestrian,40 to 44,Fatal,NaN,South,Other,NaN,NaN,NaN,Pedestrian hit at mid-block,"Crossing, no Traffic Control",Normal,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,124,Kennedy Park,138,Eglinton East (138),D41,10,62.0,Midnight
166,STEELES AVE E,NINTH LINE ST,Minor Arterial,Scarborough,25,43.842745,-79.224790,Mid-Block,NaN,No Control,Clear,Dark,Wet,Fatal,Approaching,Driver,50 to 54,Fatal,NaN,West,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,Morningside Heights,131,Rouge (131),D42,167,5.0,Night
170,STEELES AVE E,NINTH LINE ST,Minor Arterial,Scarborough,25,43.842745,-79.224790,Mid-Block,NaN,No Control,Clear,Dark,Wet,Fatal,Approaching,Vehicle Owner,unknown,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,Morningside Heights,131,Rouge (131),D42,171,5.0,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12442,BIRMINGHAM ST,TWENTY FIRST ST,Collector,Etobicoke York,3,43.599963,-79.521175,Intersection,NaN,Pedestrian Crossover,Clear,Daylight,Dry,Fatal,Sideswipe,Driver,85 to 89,Fatal,9.0,West,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,18,New Toronto,18,New Toronto (18),D22,12443,3735.0,Morning
12449,BIRMINGHAM ST,TWENTY FIRST ST,Collector,Etobicoke York,3,43.599963,-79.521175,Intersection,NaN,Pedestrian Crossover,Clear,Daylight,Dry,Fatal,Sideswipe,Driver,30 to 34,Minimal,NaN,West,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,18,New Toronto,18,New Toronto (18),D22,12450,3735.0,Morning
12485,WEDGEWOOD DR,TOBRUK CRES,Local,North York,18,43.789603,-79.413997,Intersection,NaN,Stop Sign,Clear,Daylight,Wet,Fatal,Pedestrian Collisions,Driver,45 to 49,Minimal,NaN,East,"Automobile, Station Wagon",Going Ahead,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,50,Newtonbrook East,50,Newtonbrook East (50),D32,12486,3736.0,Morning
12493,WEDGEWOOD DR,TOBRUK CRES,Local,North York,18,43.789603,-79.413997,Intersection,NaN,Stop Sign,Clear,Daylight,Wet,Fatal,Pedestrian Collisions,Pedestrian,80 to 84,Fatal,12.0,West,NaN,NaN,NaN,NaN,Pedestrian hit on sidewalk or shoulder,On Sidewalk or Shoulder,

In [416]:
# Removing ACCLOC column as it has large number of missing values

In [417]:
df = df.drop("ACCLOC", axis=1)

# #TRAFFCTL

In [418]:
# Checking values of TRAFFCTL

In [419]:
df.TRAFFCTL.value_counts()

No Control              5679
Traffic Signal          4891
Stop Sign                894
Pedestrian Crossover     142
Traffic Controller        45
Yield Sign                 9
Streetcar (Stop for)       9
Traffic Gate               5
School Guard               2
Police Control             2
Name: TRAFFCTL, dtype: int64

In [420]:
df.TRAFFCTL.isna().sum()

12

In [421]:
# We can fill missing values with the most occured value

In [422]:
df['TRAFFCTL'].fillna('No Control', inplace=True)


In [423]:
df.TRAFFCTL.isna().sum()

0

# #VISIBILITY


In [424]:
df.VISIBILITY.value_counts()

Clear                     9969
Rain                      1304
Snow                       251
Other                       69
Fog, Mist, Smoke, Dust      38
Freezing Rain               27
Drifting Snow               12
Strong wind                  6
Name: VISIBILITY, dtype: int64

In [425]:
# We can impute missing values in VISIBILTY

In [426]:
df['VISIBILITY'].fillna('Clear', inplace=True)


In [427]:
df.TRAFFCTL.isna().sum()

0

# #RDFSCOND

In [428]:
# Checking values of RDFSCOND

In [429]:
df.RDSFCOND.value_counts()

Dry                     9225
Wet                     2050
Loose Snow               122
Other                    109
Slush                     76
Ice                       51
Packed Snow               35
Loose Sand or Gravel       3
Name: RDSFCOND, dtype: int64

In [430]:
df['RDSFCOND'].isna().sum()

19

In [431]:
df['RDSFCOND'].fillna('Dry', inplace=True)


# #INVTYPE

In [432]:
# Checking values of INVTYPE

In [433]:
df.INVTYPE.value_counts()

Driver                  5127
Pedestrian              1943
Vehicle Owner           1620
Passenger               1542
Cyclist                  457
Motorcycle Driver        314
Other Property Owner     255
Truck Driver             215
Other                    143
Moped Driver              21
Motorcycle Passenger      14
Driver - Not Hit          11
Wheelchair                10
In-Line Skater             4
Trailer Owner              2
Pedestrian - Not Hit       1
Witness                    1
Name: INVTYPE, dtype: int64

In [434]:
# if values in INVTYPE is not of Fatal ACCCLASS then we can remove those records

In [435]:
df[(df["INVTYPE"].isna()) & (df["ACCLASS"] == "Fatal")]

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
9560,98 HAVENDALE RD,NA,Collector,Scarborough,22,43.792746,-79.284488,Mid-Block,No Control,Clear,Dark,Dry,Fatal,SMV Other,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,128,Agincourt South-Malvern West,128,Agincourt South-Malvern West (128),D42,9561,3121.0,Night
10446,CHILTERN HILL RD,EGLINTON AVE W,Major Arterial,North York,812,43.700518,-79.427753,Intersection,Stop Sign,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,106,Humewood-Cedarvale,106,Humewood-Cedarvale (106),D13,10447,3292.0,Morning
14021,LESLIE ST,EGLINTON AVE E,Major Arterial,North York,1516,43.718589,-79.350383,Intersection,Stop Sign,Clear,Daylight,Dry,Fatal,Turning Movement,NaN,unknown,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,Banbury-Don Mills,42,Banbury-Don Mills (42),D33,14022,3891.0,Afternoon
14096,TWENTY SIXTH ST,LABURNHAM AVE,Local,Etobicoke York,3,43.599353,-79.525653,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,NaN,unknown,NaN,NaN,NaN,"Truck - Closed (Blazer, etc)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,Long Branch,19,Long Branch (19),D22,14097,3892.0,Afternoon
14662,23 IANHALL RD,NA,Local,North York,6,43.727904,-79.488223,Mid-Block,No Control,Snow,Daylight,Loose Snow,Fatal,Pedestrian Collisions,NaN,unknown,None,NaN,East,"Automobile, Station Wagon",Going Ahead,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,Downsview,26,Downsview-Roding-CFB (26),D31,14663,3858.0,Afternoon


In [436]:
 # It has values where with Fatal ACCLASS

In [437]:
df['INVTYPE'].fillna('Driver', inplace=True)


In [438]:
df.INVTYPE.isna().sum()

0

# #INVAGE             

In [439]:
# Chcking value of INVAGE

In [440]:
df.INVAGE.value_counts()

unknown     2339
20 to 24     994
25 to 29     984
40 to 44     852
35 to 39     811
50 to 54     801
45 to 49     797
30 to 34     790
55 to 59     630
15 to 19     507
60 to 64     480
65 to 69     371
70 to 74     318
75 to 79     268
80 to 84     209
85 to 89     146
10 to 14     134
5 to 9       117
0 to 4        90
90 to 94      43
Over 95        9
Name: INVAGE, dtype: int64

In [441]:
# Age columns is important and has large number of unknown values

In [442]:
# we will do one label encoding for age columns and then drop sep

In [443]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform 'INVAGE' column
df['INVAGE'] = label_encoder.fit_transform(df['INVAGE'])

df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,None,NaN,South,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1,60.0,Morning
1,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,13,Fatal,NaN,North,Other,NaN,NaN,NaN,Vehicle turns left while ped crosses with ROW ...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2,60.0,Morning
2,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,8,Fatal,NaN,East,Motorcycle,Turning Right,Disobeyed Traffic Control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3,61.0,Morning
3,WOODBINE AVE,O CONNOR DR,Major Arterial,Toronto and East York,19,43.699595,-79.318797,Intersection,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,10,Major,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4,0.0,Midnight
4,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,20,None,NaN,South,"Automobile, Station Wagon",Going Ahead,Driving Properly,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5,61.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,NaN,NaN,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening
14964,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,NaN,NaN,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14965,3880.0,Evening
14967,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,11,None,NaN,West,"Automobile, Station Wagon",Going Ahead,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14968,3880.0,Evening
14980,ST CLAIR AVE E,JEANETTE ST,Minor Arterial,Scarborough,20,43.719565,-79.247051,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,12,Fatal,49.0,North,NaN,NaN,NaN,NaN,Vehicle is going straight thru inter.while ped...,Cro

# #INJURY

In [444]:
df.INJURY.value_counts()

None       4187
Major      3589
Minor       908
Minimal     705
Fatal       701
Name: INJURY, dtype: int64

In [445]:
df.INJURY.isna().sum()

1600

In [446]:
# Checking whether those columns has accident class of FATAL

In [447]:
df[(df["INJURY"].isna()) & (df["ACCLASS"] == "Fatal")]


,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
170,STEELES AVE E,NINTH LINE ST,Minor Arterial,Scarborough,25,43.842745,-79.224790,Mid-Block,No Control,Clear,Dark,Wet,Fatal,Approaching,Vehicle Owner,20,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,Morningside Heights,131,Rouge (131),D42,171,5.0,Night
231,WESTWOOD AVE,WOODYCREST AVE,Local,Toronto and East York,14,43.687345,-79.344690,Intersection,Stop Sign,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Vehicle Owner,20,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,59,Danforth East York,59,Danforth East York (59),D55,232,39.0,Morning
242,NEILSON RD,MCLEVIN AVE,Minor Arterial,Scarborough,2325,43.808145,-79.219990,Mid-Block,No Control,Clear,Dusk,Dry,Fatal,Pedestrian Collisions,Vehicle Owner,20,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145,Malvern West,132,Malvern (132),D42,243,44.0,Evening
269,MARTIN GROVE RD,RACINE RD,Major Arterial,Etobicoke York,1,43.717145,-79.582590,Mid-Block,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Vehicle Owner,20,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,1,West Humber-Clairville,1,West Humber-Clairville (1),D23,270,52.0,Midnight
290,F G GARDINER XY W,PARK LAWN RD,Major Arterial,Etobicoke York,3,43.627745,-79.481490,Mid-Block,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Vehicle Owner,20,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,291,58.0,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,FINCH AVE E,SKYMARK DR,Major Arterial,North York,17,43.793151,-79.352275,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,13,NaN,NaN,South,Pick Up Truck,Turning Left,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,47,Don Valley Village,47,Don Valley Village (47),D33,12061,3773.0,Morning
12061,FINCH AVE E,SKYMARK DR,Major Arterial,North York,17,43.793151,-79.352275,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Passenger,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,47,Don Valley Village,47,Don Valley Village (47),D33,12062,3773.0,Morning
14096,TWENTY SIXTH ST,LABURNHAM AVE,Local,Etobicoke York,3,43.599353,-79.525653,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,NaN,NaN,NaN,"Truck - Closed (Blazer, etc)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,Long Branch,19,Long Branch (19),D22,14097,3892.0,Afternoon
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,NaN,NaN,East,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening


In [448]:
# We can remove those with Non-Fatal Injury

In [449]:
drop_injury = df[(df["INJURY"].isna()) & (df["ACCLASS"] == "Non-Fatal Injury")].index

df = df.drop(drop_injury, axis=0)


In [450]:
# For the remaininig null values we will fill it with the most accured values

In [451]:
df.INJURY.isna().sum()

271

In [452]:
df['INJURY'].fillna('None', inplace=True)

In [453]:
df.INJURY.isna().sum()


0

# #FATAL_NO

In [454]:
# Removing FATAL_NO column because it has most of the null values in it.

In [455]:
df = df.drop('FATAL_NO', axis=1)

# #INITDIR

In [456]:
df.INITDIR.value_counts()

East       2028
West       1947
South      1904
North      1846
Unknown     239
Name: INITDIR, dtype: int64

In [457]:
df.INITDIR.isna().sum()

2397

In [458]:
# We can label encode the INITDIR becasue it may determine from where the vehicle was heading that it caused and accident

In [459]:
df['INITDIR'] = label_encoder.fit_transform(df['INITDIR'])

# #VEHTYPE

In [460]:
df.VEHTYPE.value_counts()

Automobile, Station Wagon           4757
Other                               3412
Bicycle                              453
Motorcycle                           314
Municipal Transit Bus (TTC)          189
Truck - Open                         166
Pick Up Truck                         63
Passenger Van                         50
Delivery Van                          33
Taxi                                  26
Street Car                            18
Moped                                 17
Truck - Closed (Blazer, etc)          16
Truck - Dump                          15
Truck-Tractor                         13
Truck (other)                          5
Bus (Other) (Go Bus, Gray Coach)       3
Construction Equipment                 2
School Bus                             2
Police Vehicle                         2
Intercity Bus                          2
Tow Truck                              2
Fire Vehicle                           1
Truck - Tank                           1
Name: VEHTYPE, d

In [461]:
# We can do label emcoding for this column

In [462]:
df['VEHTYPE'] = label_encoder.fit_transform(df['VEHTYPE'])

In [463]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,None,2,0,Turning Left,Failed to Yield Right of Way,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1,60.0,Morning
1,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,13,Fatal,1,10,NaN,NaN,NaN,Vehicle turns left while ped crosses with ROW ...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2,60.0,Morning
2,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,8,Fatal,0,8,Turning Right,Disobeyed Traffic Control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3,61.0,Morning
3,WOODBINE AVE,O CONNOR DR,Major Arterial,Toronto and East York,19,43.699595,-79.318797,Intersection,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,10,Major,5,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4,0.0,Midnight
4,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,20,None,2,0,Going Ahead,Driving Properly,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5,61.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening
14964,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14965,3880.0,Evening
14967,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,11,None,4,0,Going Ahead,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14968,3880.0,Evening
14980,ST CLAIR AVE E,JEANETTE ST,Minor Arterial,Scarborough,20,43.719565,-79.247051,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,12,Fatal,1,24,NaN,NaN,NaN,Vehicle is going straight thru inter.while ped...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,123,Cliffcrest,123,Cliffcrest (123),D41,14981,3898.0,Morning


In [464]:
df.VEHTYPE.isna().sum()

0

In [465]:
df.VEHTYPE.value_counts()

0     4757
10    3412
24     799
1      453
8      314
9      189
21     166
12      63
11      50
4       33
16      26
15      18
7       17
19      16
20      15
23      13
18       5
2        3
3        2
14       2
17       2
6        2
13       2
5        1
22       1
Name: VEHTYPE, dtype: int64

# #MANOEUVER

In [466]:
df.MANOEUVER.isna().sum()

3974

In [467]:
df.MANOEUVER.value_counts()

Going Ahead                            3724
Turning Left                           1036
Stopped                                 432
Turning Right                           277
Slowing or Stopping                     170
Parked                                  144
Other                                   134
Changing Lanes                          133
Unknown                                  84
Reversing                                83
Overtaking                               63
Making U Turn                            59
Pulling Away from Shoulder or Curb       26
Merging                                   9
Pulling Onto Shoulder or towardCurb       9
Disabled                                  4
Name: MANOEUVER, dtype: int64

In [468]:
# We can lable encode these values it might affect the ACCLASS

In [469]:
df.MANOEUVER.isna().sum()

3974

In [470]:
df['MANOEUVER'] = label_encoder.fit_transform(df['MANOEUVER'])


In [471]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,None,2,0,13,Failed to Yield Right of Way,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1,60.0,Morning
1,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,13,Fatal,1,10,16,NaN,NaN,Vehicle turns left while ped crosses with ROW ...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2,60.0,Morning
2,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,8,Fatal,0,8,14,Disobeyed Traffic Control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3,61.0,Morning
3,WOODBINE AVE,O CONNOR DR,Major Arterial,Toronto and East York,19,43.699595,-79.318797,Intersection,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,10,Major,5,24,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4,0.0,Midnight
4,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,20,None,2,0,2,Driving Properly,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5,61.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening
14964,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14965,3880.0,Evening
14967,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,11,None,4,0,2,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14968,3880.0,Evening
14980,ST CLAIR AVE E,JEANETTE ST,Minor Arterial,Scarborough,20,43.719565,-79.247051,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,12,Fatal,1,24,16,NaN,NaN,Vehicle is going straight thru inter.while ped...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,123,Cliffcrest,123,Cliffcrest (123),D41,14981,3898.0,Morning


# #DRIVACT             

In [472]:
df.DRIVACT.value_counts()

Driving Properly                2620
Failed to Yield Right of Way     922
Lost control                     519
Improper Turn                    338
Disobeyed Traffic Control        320
Other                            303
Following too Close              184
Exceeding Speed Limit            147
Speed too Fast For Condition     106
Improper Lane Change              77
Improper Passing                  64
Wrong Way on One Way Road          8
Speed too Slow                     2
Name: DRIVACT, dtype: int64

In [473]:
# We have label encode this column as it is relevant to ACCLASS

In [474]:
df['DRIVACT'] = label_encoder.fit_transform(df['DRIVACT'])

In [475]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,None,2,0,13,3,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1,60.0,Morning
1,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,13,Fatal,1,10,16,13,NaN,Vehicle turns left while ped crosses with ROW ...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2,60.0,Morning
2,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,8,Fatal,0,8,14,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3,61.0,Morning
3,WOODBINE AVE,O CONNOR DR,Major Arterial,Toronto and East York,19,43.699595,-79.318797,Intersection,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,10,Major,5,24,16,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4,0.0,Midnight
4,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,20,None,2,0,2,1,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5,61.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,1,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening
14964,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,1,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14965,3880.0,Evening
14967,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,11,None,4,0,2,8,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163,Fort York-Liberty Village,82,Niagara (82),D14,14968,3880.0,Evening
14980,ST CLAIR AVE E,JEANETTE ST,Minor Arterial,Scarborough,20,43.719565,-79.247051,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,12,Fatal,1,24,16,13,NaN,Vehicle is going straight thru inter.while ped...,Crossing with right of way,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,123,Cliffcrest,123,Cliffcrest (123),D41,14981,3898.0,Morning


# #DRIVCOND

In [476]:
df.DRIVCOND.value_counts()

Normal                                3678
Inattentive                            854
Unknown                                632
Had Been Drinking                      109
Medical or Physical Disability         103
Ability Impaired, Alcohol Over .08      85
Ability Impaired, Alcohol               77
Fatigue                                 31
Other                                   28
Ability Impaired, Drugs                  9
Name: DRIVCOND, dtype: int64

In [477]:
# These values also helps determine the ACCLASS, we can label encode the values

In [478]:
df['DRIVCOND'] = label_encoder.fit_transform(df['DRIVCOND'])

# #PEDTYPE

In [479]:
df.PEDTYPE.value_counts()

Pedestrian hit at mid-block                                                                     485
Vehicle turns left while ped crosses with ROW at inter.                                         382
Vehicle is going straight thru inter.while ped cross without ROW                                337
Pedestrian hit on sidewalk or shoulder                                                          141
Vehicle is going straight thru inter.while ped cross with ROW                                   119
Pedestrian involved in a collision with transit vehicle anywhere along roadway                  111
Vehicle turns right while ped crosses with ROW at inter.                                        107
Vehicle is reversing and hits pedestrian                                                         56
Other / Undefined                                                                                53
Pedestrian hit a PXO/ped. Mid-block signal                                                       42


In [480]:
# we have to label encode these values as it is important

In [481]:
df['PEDTYPE'] = label_encoder.fit_transform(df['PEDTYPE'])

# #PEDACT

In [482]:
df.PEDACT.value_counts()

Crossing with right of way               612
Crossing, no Traffic Control             446
Crossing without right of way            249
Running onto Roadway                     165
On Sidewalk or Shoulder                  161
Other                                    127
Crossing, Pedestrian Crossover            43
Person Getting on/off Vehicle             29
Coming From Behind Parked Vehicle         22
Walking on Roadway with Traffic           21
Playing or Working on Highway             20
Walking on Roadway Against Traffic        17
Pushing/Working on Vehicle                11
Crossing marked crosswalk without ROW      9
Person Getting on/off School Bus           2
Name: PEDACT, dtype: int64

In [483]:
# This values ar important in predicting the ACCLASS

In [484]:
df['PEDACT'] = label_encoder.fit_transform(df['PEDACT'])

In [485]:
df.PEDACT.isna().sum()

0

# #PEDCOND             

In [486]:
df.PEDCOND.value_counts()

Normal                                1028
Inattentive                            317
Unknown                                313
Had Been Drinking                      150
Medical or Physical Disability          50
Ability Impaired, Alcohol               30
Other                                   26
Ability Impaired, Alcohol Over .80       7
Ability Impaired, Drugs                  5
Fatigue                                  1
Name: PEDCOND, dtype: int64

In [487]:
# Need to label encode theese values as it is important

In [488]:
df['PEDCOND'] = label_encoder.fit_transform(df['PEDCOND'])

# #CYCLISTYPE

In [489]:
df.CYCLISTYPE.value_counts()

Motorist turned left across cyclists path.                                                       68
Cyclist without ROW rides into path of motorist at inter, lnwy, dwy-Cyclist not turn.            63
Cyclist and Driver travelling in same direction. One vehicle sideswipes the other.               58
Cyclist struck opened vehicle door                                                               47
Motorist without ROW drives into path of cyclist at inter, lnwy, dwy-Driver not turn.            46
Cyclist and Driver travelling in same direction. One vehicle rear-ended the other.               37
Motorist turns right at non-signal Inter.(stop, yield, no cont.,and dwy) and strikes cyclist.    23
Cyclist turned left across motorists path.                                                       19
Motorist turning right on green or amber at signalized intersection strikes cyclist.             18
Cyclist strikes pedestrian.                                                                      15


In [490]:
# We can remove these column as it contains very less values

In [491]:
df.drop('CYCLISTYPE', axis=1, inplace=True)

# #CYCACT

In [492]:
df.CYCACT.value_counts()

Driving Properly                246
Disobeyed Traffic Control        48
Other                            45
Failed to Yield Right of Way     38
Lost control                     17
Improper Passing                 14
Improper Turn                    12
Improper Lane Change             11
Speed too Fast For Condition      5
Following too Close               2
Wrong Way on One Way Road         2
Name: CYCACT, dtype: int64

In [493]:
# Have to remove this column as it is containing very less values

In [494]:
df.drop('CYCACT', axis=1, inplace=True)

# # CYCCOND

In [495]:
df.CYCCOND.value_counts()

Normal                                303
Inattentive                            60
Unknown                                48
Had Been Drinking                      16
Other                                   5
Ability Impaired, Drugs                 2
Fatigue                                 2
Ability Impaired, Alcohol Over .80      1
Ability Impaired, Alcohol               1
Medical or Physical Disability          1
Name: CYCCOND, dtype: int64

In [496]:
df.drop('CYCCOND', axis=1, inplace=True)

# We can lable encode these columns as it contains two values only
* PEDESTRIAN
* CYCLIST
* AUTOMOBILE
* MOTORCYCLE
* TRUCK
* TRSN_CITY_VEH
* EMERG_VEH
* PASSENGER
* SPEEDING
* AG_DRIV
* REDLIGHT
* ALCOHOL

In [497]:
columns_to_encode = ['PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
                     'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER', 'SPEEDING',
                     'AG_DRIV', 'REDLIGHT', 'ALCOHOL']

df[columns_to_encode] = df[columns_to_encode].apply(lambda col: label_encoder.fit_transform(col))


# #DISABILITY

In [498]:
df.DISABILITY.value_counts()

Yes    280
Name: DISABILITY, dtype: int64

In [499]:
# Have to remove this column becuase nearly all of the values are null

In [500]:
df.drop('DISABILITY', axis=1, inplace=True)

In [501]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,20,None,2,0,13,3,9,16,15,10,0,1,0,1,1,1,1,1,1,0,1,1,88,High Park North,88,High Park North (88),D11,1,60.0,Morning
1,BLOOR ST W,DUNDAS ST W,Major Arterial,Toronto and East York,4,43.656345,-79.452490,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,13,Fatal,1,10,16,13,10,12,2,9,0,1,0,1,1,1,1,1,1,0,1,1,88,High Park North,88,High Park North (88),D11,2,60.0,Morning
2,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,8,Fatal,0,8,14,0,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,146,Malvern East,132,Malvern (132),D42,3,61.0,Morning
3,WOODBINE AVE,O CONNOR DR,Major Arterial,Toronto and East York,19,43.699595,-79.318797,Intersection,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,10,Major,5,24,16,13,10,16,15,10,1,1,0,1,1,1,1,0,0,0,1,0,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4,0.0,Midnight
4,MORNINGSIDE AVE,SHEPPARD AVE E,Major Arterial,Scarborough,25,43.801943,-79.199786,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,20,None,2,0,2,1,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,146,Malvern East,132,Malvern (132),D42,5,61.0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,163,Fort York-Liberty Village,82,Niagara (82),D14,14961,3880.0,Evening
14964,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,5,None,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,163,Fort York-Liberty Village,82,Niagara (82),D14,14965,3880.0,Evening
14967,LAKE SHORE BLVD W,BRITISH COLUMBIA RD,Major Arterial,North York,10,43.630515,-79.427158,Mid-Block,No Control,Clear,Daylight,Dry,Fatal,Approaching,Driver,11,None,4,0,2,8,5,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,163,Fort York-Liberty Village,82,Niagara (82),D14,14968,3880.0,Evening
14980,ST CLAIR AVE E,JEANETTE ST,Minor Arterial,Scarborough,20,43.719565,-79.247051,Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,12,Fatal,1,24,16,13,10,9,2,9,0,1,0,1,1,1,1,1,1,0,0,1,123,Cliffcrest,123,Cliffcrest (123),D41,14981,3898.0,Morning


In [502]:
# We can remove NEIGHBOURHOOD_158 NEIGHBOURHOOD_140 as it has duplicate values with similar columns

In [503]:
df.drop(columns=['NEIGHBOURHOOD_140','NEIGHBOURHOOD_158'], axis=1, inplace=True)

# We'll need to convert string values to integer and best way to do is to label encode the data

In [504]:
columns_to_encode_2 = ['STREET1', 'STREET2', 'ROAD_CLASS','LATITUDE','LONGITUDE', 'DISTRICT', 'LOCCOORD', 'TRAFFCTL','HOOD_140','HOOD_158','VISIBILITY', 'LIGHT', 'RDSFCOND', 'ACCLASS','IMPACTYPE','INVTYPE','INJURY','TIME_SEGMENT','DIVISION']

df[columns_to_encode_2] = df[columns_to_encode_2].apply(lambda columns: label_encoder.fit_transform(columns))

In [505]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,HOOD_158,HOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,447,537,4,3,4,481,837,3,8,0,4,0,0,4,1,20,4,2,0,13,3,9,16,15,10,0,1,0,1,1,1,1,1,1,0,1,1,146,127,0,1,60.0,3
1,447,537,4,3,4,481,837,3,8,0,4,0,0,4,10,13,0,1,10,16,13,10,12,2,9,0,1,0,1,1,1,1,1,1,0,1,1,146,127,0,2,60.0,3
2,903,1586,4,2,25,2193,2661,3,8,0,4,0,0,9,5,8,0,0,8,14,0,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,45,37,10,3,61.0,3
3,1189,1261,4,3,19,1096,2032,3,0,0,0,7,1,1,9,10,1,5,24,16,13,10,16,15,10,1,1,0,1,1,1,1,0,0,0,1,0,120,97,15,4,0.0,2
4,903,1586,4,2,25,2193,2661,3,8,0,4,0,0,9,1,20,4,2,0,2,1,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,45,37,10,5,61.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,5,4,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14961,3880.0,1
14964,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,5,4,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14965,3880.0,1
14967,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,11,4,4,0,2,8,5,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14968,3880.0,1
14980,1079,930,5,2,20,1326,2492,3,8,0,4,0,0,4,10,12,0,1,24,16,13,10,9,2,9,0,1,0,1,1,1,1,1,1,0,0,1,25,27,9,14981,3898.0,3


In [506]:
df.dtypes

STREET1            int64
STREET2            int64
ROAD_CLASS         int64
DISTRICT           int64
WARDNUM            int64
LATITUDE           int64
LONGITUDE          int64
LOCCOORD           int64
TRAFFCTL           int64
VISIBILITY         int64
LIGHT              int64
RDSFCOND           int64
ACCLASS            int64
IMPACTYPE          int64
INVTYPE            int64
INVAGE             int64
INJURY             int64
INITDIR            int64
VEHTYPE            int64
MANOEUVER          int64
DRIVACT            int64
DRIVCOND           int64
PEDTYPE            int64
PEDACT             int64
PEDCOND            int64
PEDESTRIAN         int64
CYCLIST            int64
AUTOMOBILE         int64
MOTORCYCLE         int64
TRUCK              int64
TRSN_CITY_VEH      int64
EMERG_VEH          int64
PASSENGER          int64
SPEEDING           int64
AG_DRIV            int64
REDLIGHT           int64
ALCOHOL            int64
HOOD_158           int64
HOOD_140           int64
DIVISION           int64


In [507]:
df["ACC_NUM"] = df["ACC_NUM"].astype('int64')

In [508]:
df.dtypes

STREET1          int64
STREET2          int64
ROAD_CLASS       int64
DISTRICT         int64
WARDNUM          int64
LATITUDE         int64
LONGITUDE        int64
LOCCOORD         int64
TRAFFCTL         int64
VISIBILITY       int64
LIGHT            int64
RDSFCOND         int64
ACCLASS          int64
IMPACTYPE        int64
INVTYPE          int64
INVAGE           int64
INJURY           int64
INITDIR          int64
VEHTYPE          int64
MANOEUVER        int64
DRIVACT          int64
DRIVCOND         int64
PEDTYPE          int64
PEDACT           int64
PEDCOND          int64
PEDESTRIAN       int64
CYCLIST          int64
AUTOMOBILE       int64
MOTORCYCLE       int64
TRUCK            int64
TRSN_CITY_VEH    int64
EMERG_VEH        int64
PASSENGER        int64
SPEEDING         int64
AG_DRIV          int64
REDLIGHT         int64
ALCOHOL          int64
HOOD_158         int64
HOOD_140         int64
DIVISION         int64
ObjectId         int64
ACC_NUM          int64
TIME_SEGMENT     int64
dtype: obje

# Code above this SECTION

In [509]:
# First result: Total null values for each feature
null_counts = df.isnull().sum()
null_df = pd.DataFrame(null_counts, columns=["Missing Values"])

# Second result: Total unique values for each feature
unique_values = df.nunique().reset_index()
unique_values.columns = ["Feature", "Unique Values"]

# Combine the two DataFrames
combined_df = pd.merge(unique_values, null_df, left_on="Feature", right_index=True)

# Display the combined DataFrame
print(combined_df)

          Feature  Unique Values  Missing Values
0         STREET1           1202               0
1         STREET2           1954               0
2      ROAD_CLASS              8               0
3        DISTRICT              4               0
4         WARDNUM             66               0
5        LATITUDE           2361               0
6       LONGITUDE           2759               0
7        LOCCOORD              7               0
8        TRAFFCTL             10               0
9      VISIBILITY              8               0
10          LIGHT              9               0
11       RDSFCOND              8               0
12        ACCLASS              2               0
13      IMPACTYPE             10               0
14        INVTYPE             16               0
15         INVAGE             21               0
16         INJURY              5               0
17        INITDIR              6               0
18        VEHTYPE             25               0
19      MANOEUVER   

In [510]:
df.shape

(10361, 43)

# Now that we have cleaned dataset, we will check the datatype that supports best with models we are going to train on.

In [511]:
df

,STREET1,STREET2,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,HOOD_158,HOOD_140,DIVISION,ObjectId,ACC_NUM,TIME_SEGMENT
0,447,537,4,3,4,481,837,3,8,0,4,0,0,4,1,20,4,2,0,13,3,9,16,15,10,0,1,0,1,1,1,1,1,1,0,1,1,146,127,0,1,60,3
1,447,537,4,3,4,481,837,3,8,0,4,0,0,4,10,13,0,1,10,16,13,10,12,2,9,0,1,0,1,1,1,1,1,1,0,1,1,146,127,0,2,60,3
2,903,1586,4,2,25,2193,2661,3,8,0,4,0,0,9,5,8,0,0,8,14,0,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,45,37,10,3,61,3
3,1189,1261,4,3,19,1096,2032,3,0,0,0,7,1,1,9,10,1,5,24,16,13,10,16,15,10,1,1,0,1,1,1,1,0,0,0,1,0,120,97,15,4,0,2
4,903,1586,4,2,25,2193,2661,3,8,0,4,0,0,9,1,20,4,2,0,2,1,9,16,15,10,1,1,0,0,1,1,1,1,1,0,0,1,45,37,10,5,61,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14960,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,5,4,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14961,3880,1
14964,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,5,4,0,0,2,1,7,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14965,3880,1
14967,801,260,4,1,10,121,1093,4,0,0,4,0,0,1,1,11,4,4,0,2,8,5,16,15,10,1,1,0,0,1,1,1,1,1,1,1,1,64,121,3,14968,3880,1
14980,1079,930,5,2,20,1326,2492,3,8,0,4,0,0,4,10,12,0,1,24,16,13,10,9,2,9,0,1,0,1,1,1,1,1,1,0,0,1,25,27,9,14981,3898,3


## Train test split the data for traning model on algorithsm

# Decision Tree

In [512]:
x = df.drop('ACCLASS', axis=1)
y = df['ACCLASS']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

model_tree = DecisionTreeClassifier(random_state=42)

model_tree.fit(x_train, y_train)

pred_tree = model_tree.predict(x_test)

accuracy_tree = accuracy_score(y_test, pred_tree)

print(accuracy_tree)
print(classification_report(y_test, pred_tree))

0.9078630004823927
              precision    recall  f1-score   support

           0       0.77      0.78      0.77       418
           1       0.94      0.94      0.94      1655

    accuracy                           0.91      2073
   macro avg       0.86      0.86      0.86      2073
weighted avg       0.91      0.91      0.91      2073



# Logistic Regression

In [513]:
model_reg = LogisticRegression()

model_reg.fit(x_train,y_train)

pred_reg = model_reg.predict(x_test)

accuracy_reg = accuracy_score(y_test, pred_reg)

print(accuracy_reg)
print(classification_report(y_test, pred_reg))

0.8195851423058369
              precision    recall  f1-score   support

           0       0.72      0.17      0.28       418
           1       0.82      0.98      0.90      1655

    accuracy                           0.82      2073
   macro avg       0.77      0.58      0.59      2073
weighted avg       0.80      0.82      0.77      2073



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# RandomForest

In [514]:
model_random = RandomForestClassifier()

model_random.fit(x_train,y_train)

pred_random = model_random.predict(x_test)

accuracy_random = accuracy_score(y_test, pred_reg)

print(accuracy_reg)
print(classification_report(y_test,pred_reg))

0.8195851423058369
              precision    recall  f1-score   support

           0       0.72      0.17      0.28       418
           1       0.82      0.98      0.90      1655

    accuracy                           0.82      2073
   macro avg       0.77      0.58      0.59      2073
weighted avg       0.80      0.82      0.77      2073



In [515]:
model_xg = xgb.XGBClassifier(objective="binary:logistic")

model_xg.fit(x_train, y_train)

pred_xg = model_xg.predict(x_test)

accuracy_xg = accuracy_score(y_test, pred_xg)

print(accuracy_xg)
print(classification_report(y_test, pred_xg))

0.9585142305836951
              precision    recall  f1-score   support

           0       0.98      0.81      0.89       418
           1       0.95      1.00      0.97      1655

    accuracy                           0.96      2073
   macro avg       0.97      0.90      0.93      2073
weighted avg       0.96      0.96      0.96      2073



# Importing the test dataset and again training the the Train dataset and testing it on Test dataset.


